# Basic usage of the DFSZforest package
© Johannes Diehl

Compute anomaly ratios for different DFSZ-type models. This code has been used for the paper [*DFSZ axions and where to find them*](https://arxiv.com) by [Emmanouil Koutsangelas]() and [Johannes Diehl](https://scholar.google.com/citations?hl=en&user=BrHSTFwAAAAJ). The data files included can also be found separately on [Zenodo](https://zenodo.org).

In [3]:
using LinearAlgebra, StaticArrays
using Random, Statistics, StatsBase
using BenchmarkTools
using Base.Threads
using Plots
using Random: default_rng
using Symbolics
using Combinatorics
using FileIO, JLD2, HDF5
using LaTeXStrings, Printf

include("./src/drawer.jl")
include("./src/helpers.jl")

@variables u1::Int u2::Int u3::Int d1::Int d2::Int d3::Int l1::Int l2::Int l3::Int ;

## Understanding a single Yukawa sector

Let us first understand how to handle one Yukawa sector on its own, before moving on to calculate anomaly ratios for many different models in bulk.

Start by defining a Yukawa sector:

In [6]:
yukawa = [u1,u1,u3,d1,d1,d3,l1,l1,l1];

We have just defined one specific Yukawa sector with five Higgs doublets $n_D=5$.
To calculate anomaly ratios we need to specify the explicit symmetry breaking potential. To do this we need bilinears and quadrilinears. We get them using

In [12]:
bilins = get_bilins(yukawa)

20-element Vector{Num}:
  u1 - u3
  d1 + u1
  d3 + u1
  l1 + u1
  d1 + u3
  d3 + u3
  l1 + u3
  d1 - d3
  d1 - l1
  d3 - l1
  u3 - u1
 -d1 - u1
 -d3 - u1
 -l1 - u1
 -d1 - u3
 -d3 - u3
 -l1 - u3
  d3 - d1
  l1 - d1
  l1 - d3

In [10]:
quads, qmultis = get_quads(yukawa)

(Num[l1 + 2u1 - u3, 2d3 + 2u1, l1 + u3 - d1 - u1, 2l1 - 2d1, u3 + 2d1 - d3, u3 + 2d3 - l1, d1 + 2u3 - u1, d1 + l1 + u3 - d3, l1 + 2u3 - u1, 2l1 + 2u1  …  u1 + 2d3 - d1, d3 + l1 + 2u3, 2d3 + 2u3, d3 + l1 + u1 + u3, 2l1 - 2d3, d1 + l1 + u1 + u3, u1 + u3 + 2l1, d3 + 2u1 - u3, d1 + d3 + u1 - l1, u1 + 2d1 - l1], [1, 4, 2, 4, 1, 1, 1, 2, 1, 4  …  1, 1, 4, 2, 4, 2, 1, 1, 2, 1])

What we got here is not potential terms, but rather the conditions on the fermion/ Higgs doublet charges corresponding to the specific potential terms. Note that while in the definition of the yukawa sector e.g. `u1` stood for the Higgs doublet, i.e. H<sub>u<sub>1</sub></sub>, it now stands for the respective charge. So `u1` now means $\chi$<sub>u<sub>1</sub></sub>.

We use these conditions together with the orthogonality relation to find which combination of conditions allows for which values in PQ charges. Let us assume we found `u1 = 1`, `u3 = 3.5`, `d1 = 2`, `d3 = 0` and `l1 = 1`. We can then calculate the anomalies as well as the anomaly ratio.

In [35]:
Nf = get_Nfunc(yukawa)
Nf([1,1.5,2,0,1]) == N([1,1,1.5,2,2,0,1,1,1]...) == 3.75

true

In [36]:
EoNf = get_EoNfunc(yukawa)
EoNf([1,1.5,2,0,1]) == EoverN([1,1,1.5,2,2,0,1,1,1]...) == 2.4

true

But hold up! If `N = 3.75` and $N_{DW} = 2 N$, then $N_{DW} = 7.5$. This is impossible! $N_{DW}$ has to be integer! So what is going on?

If we write `u1 = 1` this implies units of the charge of the Higgs singlet $\chi_s$. The anomaly ratio itself is independent of $\chi_s$, but $E$ or $N$ are not. We therefore see that in our little example $\chi_s = 1$ is impossible, $\chi_s = 2$ has to be chosen!

## Calculating all $n_D = 4$ models

We can now move on to actually doing the calculation of the charges!

In [44]:
dataset = "test5";

The code below will compute all models for $n_D = 4$ and write the results into a h5 and a human readable textfile like the ones provided with the package. If the folder `dataset` does not exist, it will create a new one. Execution should take O(seconds).

In [46]:
models, mmultis = model_list(nD=[4])
runDFSZ_saveall(dataset, models; model_multiplicity=mmultis);
h5totxt(filename; folder="./data/DFSZ_models/"*dataset*"/")

Num[u1, u1, u3, d1, d1, d1, l1, l1, l1]


┌ Info: Total number of models to be calculated above 2520
└ @ Main /home/th347/diehl/Documents/2111-DFSZ/src/helpers.jl:920


ErrorException: cannot create dataset: object "n4_u1_u1_u3_d1_d1_d1_l1_l1_l1/bl=u1 - u3/Chis" already exists at /